In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Broadcast and Distribution Analysis

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns
import re

In [ ]:
df_guide = pd.read_csv('/kaggle/input/doctor-who/dwguide.csv')
df_guide.head(10)

In [ ]:
print('Rows:',df_guide.shape[0])
print('Columns:', df_guide.shape[1])

In [ ]:
df_guide.info()

In [ ]:
# Missing data

df_guide.isnull().sum()

In [ ]:
df_guide['AI'].fillna(method='ffill',inplace=True)

df_guide["summary"] = df_guide['summary'].fillna('No information')

In [ ]:
# Clear data

df_guide["broadcastdate"] = pd.to_datetime(df_guide["broadcastdate"],format="%d %b %Y")
df_guide["broadcasthour"] = pd.to_datetime(df_guide["broadcasthour"], 
                                           format="%I:%M%p").dt.time

df_guide['views'] = df_guide['views'].str.replace('m','').astype(np.float64)
df_guide["share"] = df_guide["share"].str.replace('%','').astype(np.float64)
df_guide["share"].fillna(0, inplace=True)

df_guide = df_guide.sort_values(by="broadcastdate", ascending=True)

In [ ]:
df_guide.head(10)

In [ ]:
# The Doctor

df_guide["Dr.Who"] = np.nan
for name in df_guide["cast"].values:
    if re.search('[{]', str(name)):
        idx = df_guide[df_guide["cast"] == name].index.values
        name_doctor = str(name).split(sep=":")[2].split("}")[0]
        df_guide.loc[idx, "Dr.Who"] = str(name_doctor)
df_guide = df_guide.drop(['cast'],axis=1)


In [ ]:
# Writers
df_guide["Writer"] = np.nan
for write in df_guide["crew"].values:
    if re.search('[{]', str(name)):
        idx = df_guide[df_guide["crew"] == write].index.values
        name_writer = str(write).split(sep=":")[2].split("}")[0]
        df_guide.loc[idx, "Writer"] = str(name_writer)
df_guide = df_guide.drop(["crew"],axis=1)

In [ ]:
# Modern serie
df_guide["ModernAge"] = df_guide["broadcastdate"].dt.year.apply(lambda x:int(x>=2005))

In [ ]:
df_guide.head(10)

In [ ]:
#Classic Doctors

classic = df_guide[df_guide["ModernAge"] == 0]
classic["Dr.Who"].unique()

In [ ]:
# Modern Doctors
modern = df_guide[df_guide["ModernAge"] == 1]
modern["Dr.Who"].unique()

In [ ]:
# Number of episodes for each Doctor
doctors_episodes = df_guide["Dr.Who"].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(y=doctors_episodes.index, x=doctors_episodes)
plt.xlabel('Number of episodes for each Doctor')
plt.show()


In [ ]:
#Appreciation index for each Doctor
AI_doctors = df_guide.groupby(by="Dr.Who").mean()["AI"].sort_values(ascending=False)
plt.figure(figsize=(12,6))
sns.barplot(y=AI_doctors.index, x=AI_doctors)
plt.xlabel('Appreciation index for each Doctor')
plt.show()

In [ ]:
#Number of views of the first broadcast (millions) for each Doctor
number_views = df_guide.groupby(by="Dr.Who").mean()["views"].sort_values(ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(y=number_views.index, x=number_views)
plt.xlabel('Number of views of the first broadcast (millions) for each Doctor')
plt.show()

In [ ]:
# Number of episodes for each writer
writer = df_guide["Writer"].value_counts()

plt.figure(figsize=(12,9))
plt.barh(writer.index[:30], writer[:30])
plt.xlabel('30 writer with more number of episodes')
plt.show()

In [ ]:
# Appreciation index for each writer
AI_writer = df_guide.groupby(by="Writer").mean()["AI"].sort_values(ascending=False)
AI_writer

In [ ]:
#Number of views of the first broadcast (millions) for each writer
c =df_guide.groupby(by="Writer").mean()["views"].sort_values(ascending=False)

plt.figure(figsize=(12,9))
plt.barh(c.index[:30], c[:30])
plt.xlabel('30 writer with more number of views of the first broadcast (millions)')
plt.show()